This program creates box plots for the output from the TIME web application's Workflow 5b, which caluclates uses their custom Dynamic Time Warping (DTW) algorithm to calculate the pairwise DTW distance between bacterial taxa. For the CF data, I used the following settings: a taxonomic level of 'Genus,' the default constraint of 2 and a filter for rare taxa of 0.1 (because the input data was already filtered). The input file was not rarefied or normalised in the intitial upload. In the programs Create_TDTW_all_filtered and Create_TDTW_all_example I created csv files for Workflow 5b output data. Now it's time to plot them, with a specific focus on 3 bacteria of particular interest to our CF research. 

As with my other programs, this code may easily be adapted to create boxplots for other types of data. It plots a selection of columns from the table rather than grouping by a factor variable. It also extracts names of bacteria from much longer strings. See the program DTW_All_boxplots_example for the same code using the example file generated in Create_TDTW_all_example, which you can run for yourself.

In [1]:
import pandas as pd
import numpy as np

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [3]:
#read in the file, created in the program Create_TDTW_all_filtered, which contains all the output from Workflow 5b
df = pd.read_csv(r"F:\CF\Data\TIME\DTW\TDTW_all_filtered.csv")

The first few cells modify the dataset to focus on only 3 taxa, creating a separate file for each of the three which shows the distance from the other bacteria to that one bacteria. 

<br>

If the filtered files have already been created and saved for the bacteria you wish to plot, there are alternate versions of the code for both sets of plots that reads them in to plot them.

In [ ]:
#rename the first column, which has no name, to 'Sample' 
df.rename(columns={'Unnamed: 0': 'Sample'}, inplace=True)


In [ ]:
#extract the participant's clinical status from the sample identifier, and change to title case
df['Status']=df['Sample'].str.split('_').str[2].str.title()


In [ ]:
#extract the participant's ID number from the identifier
df['ID']=df['Sample'].str.split('_').str[1]
#review the changes to confirm that they worked
df.head()

What the new columns should look like:
<img src='https://imgur.com/KU0nqUf.png' style='height:175px'>

In [ ]:
#create a separate data frame for just those entries with status 'All'
df2=df[df['Status'] == 'All']
#view the head of the new table if desired
df2.head()

Note that in our CF input data for TIME, taxonomic names are preceded by double underscores at all levels.
<br>

Example: D_0__Bacteria;D_1__Actinobacteria;D_2__Actinobacteria;D_3__Actinomycetales;D_4__Actinomycetaceae;D_5__Actinomyces
<br>

This generates output for the genus level with Actinomyces preceded by double underscores in every workflow.
We left the underscores in as separators when we merged the output files from TIME in Create_TDTW_all_filtered. The example data lacked these underscores, so we chose to add a single underscore as a separator between bacteria names in Create_TDTW_all_example. This means that, if you are editing this program to run it on your own data, the code below may need to be modified to reflect any formatting differences. 

In [8]:
#make a list of the columns we are interested in, excluding those that pair OTU's with themselves 
clist=[]
bacteria=['Pseudomonas','Staphylococcus','Streptococcus']
for i in range(0,3):
    #if you are editing this code to use on your own data, here you may need a minor edit to relect formatting differences
    #viewing the head of df or df2 above should tell you if and how you need to change the str.format() function
    j=[col for col in df.columns if col.startswith('__{}'.format(bacteria[i])) 
       and col!= '__{}__{}'.format(bacteria[i],bacteria[i])]
    clist.append(j)                                              
#view the list to check it, if desired
#print(clist)


[['__Pseudomonas__Gemella', '__Pseudomonas__Veillonella', '__Pseudomonas__Prevotella 7', '__Pseudomonas__Alloprevotella', '__Pseudomonas__Actinomyces', '__Pseudomonas__Prevotella', '__Pseudomonas__Porphyromonas', '__Pseudomonas__Rothia', '__Pseudomonas__Fusobacterium', '__Pseudomonas__Staphylococcus', '__Pseudomonas__Moraxella', '__Pseudomonas__Stenotrophomonas', '__Pseudomonas__Haemophilus', '__Pseudomonas__Neisseria', '__Pseudomonas__Streptococcus', '__Pseudomonas__Granulicatella', '__Pseudomonas__Actinobacillus', '__Pseudomonas__Leptotrichia', '__Pseudomonas__Capnocytophaga'], ['__Staphylococcus__Gemella', '__Staphylococcus__Veillonella', '__Staphylococcus__Prevotella 7', '__Staphylococcus__Alloprevotella', '__Staphylococcus__Actinomyces', '__Staphylococcus__Prevotella', '__Staphylococcus__Porphyromonas', '__Staphylococcus__Rothia', '__Staphylococcus__Fusobacterium', '__Staphylococcus__Moraxella', '__Staphylococcus__Pseudomonas', '__Staphylococcus__Stenotrophomonas', '__Staphylococc

I included the code in all of my different plotting programs for GP Microbiome output the creation of keys for bacteria names. The keys pair OTU ID numbers with names of bacteria. The file 'OTU_key_named_selection.csv' contains only the bacteria of particular interest to the CF study. 'OTU_key_named_selection.xlsx' is that file, with information added about the NCBI taxonomic ID and how we want the boxplot to look, saved as an Excel workbook. 

For quick basic plots without including the NCBI taxonomic ID's or specifying the order, skip to the last two cells.

In [52]:
#read in the file and take a look at the columns
key=pd.read_excel(r"F:\CF\Data\OTU_key_named_selection.xlsx")
key.head()


,ID_OTU,Bacteria,Name,NCBI:txid,in_boxplot,order for plotting
0,OTU_94,D_0__Bacteria;D_1__Firmicutes;D_2__Bacilli;D_3...,Staphylococcus,1279,Y,0.0
1,OTU_229,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammapr...,Pseudomonas,286,Y,1.0
2,OTU_169,D_0__Bacteria;D_1__Fusobacteria;D_2__Fusobacte...,Fusobacterium,848,Y,2.0
3,OTU_206,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammapr...,Neisseria,482,Y,3.0
4,OTU_223,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammapr...,Haemophilus,724,Y,4.0


In [22]:
#create and view a dictionary for the taxonomic ID's
taxid_key=dict(zip(key['Name'], key['NCBI:txid']))
#view dictionary, if desired
taxid_key

{'Pseudomonas': 286,
 'Moraxella': 475,
 'Neisseria': 482,
 'Actinobacillus': 713,
 'Haemophilus': 724,
 'Porphyromonas': 836,
 'Prevotella': 838,
 'Fusobacterium': 848,
 'Capnocytophaga': 1016,
 'Staphylococcus': 1279,
 'Streptococcus': 1301,
 'Gemella': 1378,
 'Actinomyces': 1654,
 'Veillonella': 29465,
 'Leptotrichia': 32067,
 'Rothia': 32207,
 'Stenotrophomonas': 40323,
 'Granulicatella': 117563,
 'Alloprevotella': 1283313,
 'Prevotella 7': '838 (7)'}

In [56]:
taxa=key[key['in_boxplot']=='Y'].sort_values('order for plotting')
taxa.head()


,ID_OTU,Bacteria,Name,NCBI:txid,in_boxplot,order for plotting
0,OTU_94,D_0__Bacteria;D_1__Firmicutes;D_2__Bacilli;D_3...,Staphylococcus,1279,Y,0.0
1,OTU_229,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammapr...,Pseudomonas,286,Y,1.0
2,OTU_169,D_0__Bacteria;D_1__Fusobacteria;D_2__Fusobacte...,Fusobacterium,848,Y,2.0
3,OTU_206,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammapr...,Neisseria,482,Y,3.0
4,OTU_223,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammapr...,Haemophilus,724,Y,4.0


## Plots with Taxonomic ID's:

Loop that creates the files:

In [ ]:
for i in range(0,3):
    df3=df2.filter(clist[i], axis=1)
    #since each data frame is for the distance from one bacteria, change the column names to be just the second bacteria
    #this code may need adjustment depending on how many underscores are in your bacteria names
    df3.columns=[w.replace('__{}__'.format(bacteria[i]),'') for w in df3.columns]
    #save file if desired
    #df3.to_csv(r"F:\CF\Data\TIME\DTW\TDTW_filtered_{}.csv".format(bacteria[i]), index=False)
    #to keep those with at least 4 non-NaN's:
    #df3.dropna(thresh=4, axis=1, inplace=True)
    df3.dropna(axis=1, inplace=True)
    order=list(taxa['Name'][taxa['Name']!=bacteria[i]])
    df3=df3.reindex(columns=order)
    df3.columns=[col+' NCBI:taxid {}'.format(str(taxid_key[col])) for col in df3.columns]
    plt.figure(figsize=(12,14))
    ax=sns.boxplot(x="variable", y="value", data=pd.melt(df3))
    plt.setp(ax.get_xticklabels(), rotation=90, size=18, style='italic')
    plt.setp(ax.get_yticklabels(), size=16)
    plt.title('DTW Distance from {}'.format(bacteria[i]), size=24)
    plt.xlabel('')
    plt.ylabel('Distance',size=20)
    plt.tight_layout()
    #plt.savefig(r"F:\CF\Data\TIME\DTW\{}_ALL.png".format(bacteria[i]), format = "png")
    plt.show()

Loop that reads in the files:

In [ ]:
#loop for if the files have already been created
for i in range(0,3):
    df = pd.read_csv(r"F:\CF\Data\TIME\DTW\TDTW_filtered_{}.csv".format(bacteria[i]))
    df.dropna(axis=1, inplace=True)
    order=list(taxa['Name'][taxa['Name']!=bacteria[i]])
    df=df.reindex(columns=order)
    df.columns=[col+' NCBI:taxid {}'.format(str(taxid_key[col])) for col in df.columns]
    plt.figure(figsize=(12,14))
    ax=sns.boxplot(x="variable", y="value", data=pd.melt(df))
    plt.setp(ax.get_xticklabels(), rotation=90, size=18, style='italic')
    plt.setp(ax.get_yticklabels(), size=16)
    plt.title('DTW Distance from {}'.format(bacteria[i]), size=24)
    plt.xlabel('')
    plt.ylabel('Distance',size=20)
    plt.tight_layout()
    #plt.savefig(r"F:\CF\Data\TIME\DTW\{}_ALL.png".format(bacteria[i]), format = "png")
    plt.show() 

Quick basic plots:

In [ ]:
for i in range(0,3):
    df3=df2.filter(clist[i], axis=1)
    #since each data frame is for the distance from one bacteria, change the column names to be just the second bacteria
    #this code may need adjustment depending on how many underscores are in your bacteria names    
    df3.columns=[w.replace('__{}__'.format(bacteria[i]),'') for w in df3.columns]
    #save file if desired
    #df3.to_csv(r"F:\CF\Data\TIME\DTW\TDTW_filtered_{}.csv".format(bacteria[i]), index=False)
    #to keep those with at least 4 non-NaN's:
    #df3.dropna(thresh=4, axis=1, inplace=True)
    df3.dropna(axis=1, inplace=True)
    plt.figure(figsize=(12,10))
    ax=sns.boxplot(x="variable", y="value", data=pd.melt(df3))
    plt.setp(ax.get_xticklabels(), rotation=90, size=18, style='italic')
    plt.setp(ax.get_yticklabels(), size=16)
    plt.title('DTW Distance from {}'.format(bacteria[i]), size=24)
    plt.xlabel('')
    plt.ylabel('Distance',size=20)
    plt.tight_layout()
    #plt.savefig(r"F:\CF\Data\TIME\DTW\{}_ALL.png".format(bacteria[i]), format = "png")
    plt.show() 

In [ ]:
#loop for if the files have already been created
for i in range(0,3):
    df = pd.read_csv(r"F:\CF\Data\TIME\DTW\TDTW_filtered_{}.csv".format(bacteria[i]))
    df.dropna(axis=1, inplace=True)
    plt.figure(figsize=(12,10))
    ax=sns.boxplot(x="variable", y="value", data=pd.melt(df))
    plt.setp(ax.get_xticklabels(), rotation=90, size=18, style='italic')
    plt.setp(ax.get_yticklabels(), size=16)
    plt.title('DTW Distance from {}'.format(bacteria[i]), size=24)
    plt.xlabel('')
    plt.ylabel('Distance',size=20)
    plt.tight_layout()
    #plt.savefig(r"F:\CF\Data\TIME\DTW\{}_ALL.png".format(bacteria[i]), format = "png")
    plt.show() 